This notebook provides an example of *regridding* using `nco` and the Earth System Modeling Framework.
# Requirements
To run this notebook you need:
* a recent version of [`nco`](http://nco.sourceforge.net/) toolbox
* the Earth System Modeling Framework [ESMF](https://www.earthsystemcog.org/projects/esmf/)

# Configuration

Check if `ncremap` is installed

In [1]:
which ncremap

/usr/local/bin/ncremap


Check if the LD_LIBRARY_PATH is properly defined:    
it has to include the path to the directory containing `libnco.so`

In [2]:
echo ${LD_LIBRARY_PATH}

If that's not the case, add it like this:

In [3]:
export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/usr/local/lib
echo ${LD_LIBRARY_PATH}

:/usr/local/lib


Check if `ESMF_RegridWeightGen` is installed and in your PATH:

In [4]:
which ESMF_RegridWeightGen

/home/ctroupin/bin/ESMF_RegridWeightGen


# Input files
We will need two files:
1. A data file (`regrid_input.nc`) containing at least one field to be re-interpolated.
2. A grid file (`regrid_newgrid.nc`) storing the coordinates (longitude, latitude) of the new grid.     
These files are located in the `data` directory.

In [7]:
ls ./data

regrid_input.nc  regrid_newgrid.nc  regrid_output_ref.nc


## Modify dimension and variable names

The tool expects that the dimension are lon and lat, while our file has x and y.     
We will use [`ncrename`](https://linux.die.net/man/1/ncrename) to solve this.

In [9]:
cp ./data/regrid_input.nc ./data/regrid_input_latlon.nc
ncrename -d x,lon -v x,lon -d y,lat -v y,lat ./data/regrid_input_latlon.nc

ncrename: In total renamed 0 attributes, 2 dimensions, 0 groups, and 2 variables


### Perform the regridding

In [10]:
ncremap -i ./data/regrid_input_latlon.nc -d ./data/regrid_newgrid.nc -o ./data/regrid_output.nc

Input #00: ./data/regrid_input_latlon.nc
Grid(src): /tmp/ncremap_tmp_grd_src.nc.pid14558
Grid(dst): /tmp/ncremap_tmp_grd_dst.nc.pid14558
Map/Wgt  : /tmp/ncremap_tmp_map_esmf_bilinear.nc.pid14558
Regridded: ./data/regrid_output.nc


Now we can check the dimensions of the newly created file:

In [12]:
ncdump -h ./data/regrid_output.nc | head -5

netcdf regrid_output {
dimensions:
	lat = 321 ;
	lon = 851 ;
	nbnd = 2 ;


and compare them to the original one:

In [14]:
ncdump -h ./data/regrid_input.nc | head -5

netcdf regrid_input {
dimensions:
	x = 426 ;
	y = 161 ;
variables:


## Final check

Let's compare with the reference file provided in the directory:

In [15]:
ncdiff ./data/regrid_output.nc ./data/regrid_output_ref.nc ./data/output_diff.nc

If everything went fine, the resulting field should only contain zeros.     
We show only the first lines of the `ncdump` operation:

In [21]:
ncdump -v analyzed_field ./data/output_diff.nc | head -200 | tail -100

    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 